# Stixrude-Lithgow-Bertelloni pseudo-omnicomponent phase generation
Required Python packages/modules

In [ ]:
import numpy as np
from os import path
import pandas as pd
import scipy.optimize as opt
import scipy.linalg as lin 
import scipy as sp
import sys
import sympy as sym

import matplotlib.pyplot as plt

Required ENKI modules (ignore the error message from Rubicon running under Python 3.6+)

In [ ]:
from thermoengine import coder, core, phases, model, equilibrate

In [ ]:
def get_subsolidus_phases(database='Berman'):
    remove_phases = ['Liq','H2O']
    
    modelDB = model.Database(database)
    phases = modelDB.phases
    [phases.pop(phs) for phs in remove_phases]
        
    return phases
        
def system_energy_landscape(T, P, phases, TOL=1e-3):
    elem_comps = []
    phs_sym = []
    endmem_ids = []
    mu = []
    for phsnm in phases:
        phs = phases[phsnm]
        
        elem_comp = phs.props['element_comp']
        abbrev = phs.abbrev
        endmem_num = phs.endmember_num
        iendmem_ids = list(np.arange(endmem_num))
        
        if phs.phase_type=='pure':
            nelem = np.sum(elem_comp)
            mu += [phs.gibbs_energy(T, P)/nelem]
            # print(nelem)
        else:
            nelem = np.sum(elem_comp,axis=1)
            # print(nelem)
            for i in iendmem_ids:
                imol = np.eye(phs.endmember_num)[i]
                mu += [phs.gibbs_energy(T, P, mol=imol,deriv={"dmol":1})[0,i]/nelem[i]]
                # print(nelem[i])
                
        endmem_ids.extend(iendmem_ids)
        phs_sym.extend(list(np.tile(abbrev,endmem_num)))
        # print(elem_comp)
        
        elem_comps.extend(elem_comp)
        # print(elem_comp)
        # print(phs)
        
    elem_comps = np.vstack(elem_comps)
    
    natoms = np.sum(elem_comps,axis=1)
    elem_comps = elem_comps/natoms[:,np.newaxis]
    
    elem_mask = ~np.all(elem_comps<TOL, axis=0)
    
    elem_comps = elem_comps[:, elem_mask]
    mu = np.array(mu)
    endmem_ids = np.array(endmem_ids)
    
    sys_elems = core.chem.PERIODIC_ORDER[elem_mask]
    return phs_sym, endmem_ids, mu, elem_comps, sys_elems

def prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps, decimals=4):
    elem_round_comps = np.round(elem_comps, decimals=decimals)
        # Drop identical comps
    elem_comps_uniq = np.unique(elem_round_comps, axis=0)
    
    # uniq_num = elem_comps_uniq.shape[0]
    mu_uniq = []
    phs_sym_uniq = []
    endmem_ids_uniq = []
    for elem_comp in elem_comps_uniq:
        is_equiv_comp = np.all(elem_round_comps == elem_comp[np.newaxis,:], axis=1)
        equiv_ind = np.where(is_equiv_comp)[0]
        min_ind = equiv_ind[np.argsort(mu[equiv_ind])[0]]
        min_mu = mu[min_ind]
        assert np.all(min_mu <= mu[equiv_ind]), 'fail'
        
        mu_uniq.append(min_mu)
        phs_sym_uniq.append(phs_sym[min_ind])
        endmem_ids_uniq.append(endmem_ids[min_ind])
        
    mu_uniq = np.array(mu_uniq)
    phs_sym_uniq = np.array(phs_sym_uniq)
    elem_comps_uniq = np.array(elem_comps_uniq)
    
    return phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq



In [ ]:
def min_energy_assemblage(bulk_comp, comp, mu, TOLmu=10, TOL=1e-5):
    xy = np.hstack((comp, mu[:,np.newaxis]))
    yavg = np.mean(mu)
    xy_bulk = np.hstack((bulk_comp, yavg))
    
    wt0, rnorm0 = opt.nnls(xy.T, xy_bulk)
    # print('rnorm',rnorm0)
    
    
    def fun(mu, shift=0):
        xy_bulk[-1] = mu
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        return rnorm-shift
    
    
    delmu = .1
    if rnorm0==0:
        shift_dir = -1
        soln_found = True
    else:
        output = opt.minimize_scalar(fun, bounds=[np.min(mu), np.max(mu)])
        xy_bulk[-1] = output['x']
        wt0, rnorm0 = opt.nnls(xy.T, xy_bulk)
        shift_dir = -1
        
    mu_prev=xy_bulk[-1]
    rnorm=rnorm0
    
    while True:
        mu_prev = xy_bulk[-1]
        rnorm_prev = rnorm
        
        xy_bulk[-1] += shift_dir*delmu
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        delmu *= 2
        
        # print(shift_dir, rnorm)
        if ((shift_dir==+1)&(rnorm>rnorm_prev)) or ((shift_dir==-1)&(rnorm>0)):
            break
            
        
    fun_fit = lambda mu, TOL=TOL: fun(mu, shift=TOL)
    if rnorm > TOL:
        mu_bulk = opt.brentq(fun_fit, mu_prev, xy_bulk[-1], xtol=TOLmu)
        xy_bulk[-1] = mu_bulk
        wt, rnorm = opt.nnls(xy.T, xy_bulk)
        
    mu_bulk = xy_bulk[-1]
    wt_bulk = wt
        
        
    ind_assem = np.where(wt_bulk>0)[0]
    return wt_bulk, mu_bulk, ind_assem 


In [ ]:
def eval_curv(comps, method, cross_term_inds):
    single_pt = False
    if comps.ndim==1:
        single_pt = True
        comps = comps[np.newaxis,:]
        
    if method=='quad':
        XiXj = comps[:, cross_term_inds[0]]*comps[:, cross_term_inds[1]]
        X2_sum = np.sum(XiXj,axis=1)
        curv_term = X2_sum
    elif method=='quad-full':
        XiXj = comps[:, cross_term_inds[0]]*comps[:, cross_term_inds[1]]
        curv_term = XiXj
    elif method=='xlogx':
        logX = np.log(comps)
        logX[comps==0] = 0
        XlogX = comps*logX
        # XlogX[comps==0] = 0
        XlogX_sum = np.sum(XlogX,axis=1)
        curv_term = XlogX_sum
    elif method=='none':
        curv_term = np.zeros((comps.shape[0],0))
    else:
        assert False, method + ' is not a valid method for eval_curv.'
        
    if single_pt:
        curv_term = curv_term[0]
    
    return curv_term

In [ ]:
def init_lstsq(comps, mu, curv_method, cross_term_inds, yscl=None):
    curv_term = eval_curv(comps, curv_method, cross_term_inds)
    if curv_term.ndim==1:
        curv_term = curv_term[:,np.newaxis]
    
    print(curv_term)
        
    xobs = np.hstack((comps, curv_term))
    if yscl is None:
        yexp_scl = np.floor(np.log10(np.max(mu)-np.min(mu)))
        yscl = 10**yexp_scl
        
    yobs = mu/yscl
    
    return xobs, yobs, yscl

### T,P, parameters and options for pseudo-phase generation

In [ ]:
T = 1300.0                  # K
P = 20000.0                 # bars
P = 300000.0                 # bars
# T = 1500.0                  # K
# P = 40000.0                 # bars

In [ ]:
# database='Berman'
database='Stixrude'

In [ ]:
# ADD extra phases (e.g. carbonates as needed here)

In [ ]:
database='Stixrude'
phases = get_subsolidus_phases(database=database)
phs_sym, endmem_ids, mu, elem_comps, sys_elems = system_energy_landscape(T, P, phases)
# display(phs_sym, endmem_ids, mu, elem_comps, sys_elems)
phs_sym_uniq, endmem_ids_uniq, mu_uniq, elem_comps_uniq = (
    prune_polymorphs(phs_sym, endmem_ids, mu, elem_comps))
Nelems = len(sys_elems)
Npts = mu_uniq.size

In [ ]:
sys_elems

In [ ]:
def get_quad_inds(Nelems):
    ind_rows, ind_cols = np.tril_indices(Nelems,-1)
    cross_term_inds = np.vstack((ind_rows,ind_cols))
    return cross_term_inds

cross_term_inds = get_quad_inds(Nelems)
# cross_term_inds[0]

# Define bulk composition

In [ ]:
wt = np.random.rand(elem_comps_uniq.shape[0])
wt = wt/np.sum(wt)
bulk_comp = np.dot(wt, elem_comps_uniq)

# Get minimum energy assemblage

In [ ]:
wt_bulk, mu_bulk, ind_assem = min_energy_assemblage(
    bulk_comp, elem_comps_uniq, mu_uniq, TOLmu=10)
comp_assem, mu_assem = elem_comps_uniq[ind_assem], mu_uniq[ind_assem]
comp_assem_avg = np.mean(comp_assem,axis=0)
X2scl = np.min(np.sum((comp_assem-comp_assem_avg)**2,axis=1))

In [ ]:
comp_assem

In [ ]:
bulk_comp

In [ ]:
wt_bulk[ind_assem]

# Fit simple quadratic (diagonal terms only) as function of endmember fractions


In [ ]:


def fit_quad_excess_endmem(comp_assem, wt_bulk_assem, dmu_endmem=3e3):
    N_assem = comp_assem.shape[0]
    
    wt_endmem = np.eye(N_assem)
    dwt_endmem = wt_endmem - wt_bulk_assem
    # X_endmem = np.vstack((np.eye(N_assem), wt_bulk_assem))
    # mu_vals = np.hstack((np.tile(dmu_endmem, N_assem), dmu_bulk))
    mu_vals = np.tile(dmu_endmem, N_assem)
    
    # np.round(X_endmem,decimals=3)
    # xobs = np.hstack((dX_endmem, dX2_endmem))
    
    dwt2_endmem = dwt_endmem**2
    output = np.linalg.lstsq(dwt2_endmem, mu_vals, rcond=None)
    # output = np.linalg.lstsq(xobs, mu_vals, rcond=None)
    curv_endmem = output[0]
    
    return curv_endmem

In [ ]:
wt_bulk_assem = wt_bulk[ind_assem]
curv_endmem = fit_quad_excess_endmem(comp_assem, wt_bulk_assem, dmu_endmem=3e3)

In [ ]:
wt_bulk_assem

In [ ]:
dmu_bulk = 0.3e3
dmu_endmem = 3e3

In [ ]:
def random_sample(curv_endmem, wt_bulk_assem, Nsamp=9000):
    N_assem = len(wt_bulk_assem)
    wt_rand = np.random.rand(Nsamp, N_assem)
    wt_rand = wt_rand/np.sum(wt_rand, axis=1)[:,np.newaxis]
    
    dwt_rand = wt_rand-wt_bulk_assem
    dwt2_rand = dwt_rand**2
    
    mu_rand = np.dot(dwt2_rand, curv_endmem)
    
    
    return wt_rand, mu_rand


In [ ]:
wt_rand, mu_rand = random_sample(curv_endmem, wt_bulk_assem)
comp_rand = np.dot(wt_rand, comp_assem)

In [ ]:
def fit_quad_shift(bulk_comp, comp_rand, mu_rand, comp_assem, dmu_endmem, dmu_bulk):
    
    N_assem = comp_assem.shape[0]
    N_elems = len(bulk_comp)
    
    yobs = np.hstack((0, np.tile(dmu_endmem, N_assem), mu_rand)) + dmu_bulk
    comps = np.vstack((bulk_comp, comp_assem, comp_rand))
    
    cross_terms_ind = get_quad_inds(N_elems)
    X2 = eval_curv(comps, 'quad-full', cross_terms_ind)
    
    xobs = np.hstack((comps,X2))
    
    fit_output = np.linalg.lstsq(xobs, yobs, rcond=None)
    coef = fit_output[0]
    chem_pot = coef[:N_elems]
    quad_coef = coef[N_elems:]
    
    
    return chem_pot, quad_coef, xobs, yobs, fit_output

chem_pot, quad_coef, xobs, yobs, fit_output = fit_quad_shift(bulk_comp, comp_rand,mu_rand, comp_assem, dmu_endmem, dmu_bulk)



In [ ]:
N_assem = comp_assem.shape[0]
for ind in range(Nelems):
    plt.figure()
    plt.plot(xobs[N_assem+1:,ind],yobs[N_assem+1:], 'o',color=[.5,.5,.5] )
    plt.plot(xobs[:N_assem+1, ind],yobs[:N_assem+1], 'rx', ms=8)
    plt.plot(comp_assem[:,ind],np.zeros(N_assem), 'k--x')
    plt.plot(bulk_comp[ind],0, 'kx', mew=4,ms=10)



In [ ]:
def fit_assemblage_chem_pot(comp_assem, mu_assem):
    fit_output_assem = np.linalg.lstsq(comp_assem, mu_assem, rcond=None)
    chem_pot_assem = fit_output_assem[0]
    
    return chem_pot_assem, fit_output_assem

chem_pot_assem, fit_output_assem = fit_assemblage_chem_pot(comp_assem, mu_assem)

In [ ]:
chem_pot_assem

In [ ]:
chem_pot

In [ ]:
mu_linear = chem_pot_assem+chem_pot
mu_linear

In [ ]:
# quad_consts = quad_coef

## Build endmembers of pseudo-phase using the coder module

In [ ]:
modelCD = coder.StdStateModel()
modelCD.set_module_name('pseudo_end')

GTP = sym.symbols('GTP')
params = [('GTP','J',GTP)]
modelCD.add_expression_to_model(GTP, params)


model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
def standardize_formula(form):
    cmp = form.split('O')
    str = ''
    if cmp[0][-1].isdigit():
        str += cmp[0][:-1] + '(' + cmp[0][-1] + ')'
    else:
        str += cmp[0] + '(1)'
    if cmp[1] == '':
        str += 'O'
    else:
        str += 'O(' + cmp[1] + ')'
    return str

In [ ]:
use_oxides_as_basis = False

In [ ]:
model_type = "calib"
for ind,elm in enumerate(sys_elems):
    imu = mu_linear[ind]
    print(imu)
    if use_oxides_as_basis:
        formula = standardize_formula(elm)
    else:
        formula = elm+'(1)'
    param_dict = {'Phase':elm,'Formula':formula,'T_r':298.15,'P_r':1.0,'GTP':imu}
    print (param_dict)
    result = modelCD.create_code_module(
        phase=param_dict.pop('Phase', None),
        formula=param_dict.pop('Formula', None),
        params=param_dict, module_type=model_type, silent=True)
    print ('Component', elm, 'done!')

Build the code (ignore error messages generated by Cython regarding 'language_level')

In [ ]:
import pseudo_end
%cd ..

In [ ]:
elm_sys=sys_elems

In [ ]:
c = len(elm_sys)
c

In [ ]:
modelCD = coder.SimpleSolnModel(nc=c)

In [ ]:
n = modelCD.n
nT = modelCD.nT
X = n/nT

In [ ]:
len(X)

In [ ]:
# XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
# XiXj

In [ ]:
mu = modelCD.mu
mu

In [ ]:
# Tsym = modelCD.get_symbol_for_t()

In [ ]:
G_ss = (n.transpose()*mu)[0]
G_ss

In [ ]:
X.shape

In [ ]:
# dX = X[:,0]- bulk_comp[:,np.newaxis]
# dX

In [ ]:
# X0 = sym.MatrixSymbol('X_0',c,1)
# X0.shape

In [ ]:
# dX = (X[:,0]-X0[:,0])
# dX = X-X0
# dX.shape

In [ ]:
# sym.Matrix(dX)

In [ ]:
# if curv_method=='quad-full':

curv_string = ''
quad_strs = []
for i,j in cross_term_inds.T:
    # print(i, j)
    istr = 'k_' + str(i+1) + '_' + str(j+1)
    curv_string +=  istr + ' '
    quad_strs.append(istr)
    

quad_consts = sym.Matrix(list(sym.symbols(curv_string)))

# k_curv = sym.symbols('k_curv')
# k_curv

quad_consts

In [ ]:
# Bulk composition unnecessary

# bulk_string = ''
# bulk_strs = []
# for i in np.arange(Nelems):
#     # print(i, j)
#     istr = 'Xbulk_' + str(i+1)
#     bulk_string +=  istr + ' '
#     bulk_strs.append(istr)
#     
# 
# bulk_sym = sym.Matrix(list(sym.symbols(bulk_string)))

In [ ]:
# bulk_sym

In [ ]:
# quad_consts

In [ ]:
#dX = n/nT-bulk_sym

# X2_sym = np.dot(dX.T,dX)[0,0]
# G_local_quad = nT*k_curv*X2_sym
# G_local_quad

# XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
# G_curv = k_curv*np.sum(nT*XiXj)

In [ ]:
# quad_strs

In [ ]:
XiXj = np.dot(n,n.T)[cross_term_inds[0], cross_term_inds[1]]/nT**2
G_quad = nT*np.dot(XiXj, quad_consts)[0]
G_quad

In [ ]:
# G_quad

In [ ]:
mu_shft = sym.symbols('mu_shft')
mu_shft

In [ ]:
# Ncross_terms = cross_term_inds.shape[1]
# Ncross_terms

In [ ]:
Gshft = mu_shft*nT

In [ ]:
# G = G_ss + G_local_quad + Gshft
# G = G_ss + G_curv + Gshft
G = G_ss + G_quad + Gshft
G

In [ ]:
mu_expr = G.diff(n)
mu_expr[0]

In [ ]:
mu

In [ ]:
mu

In [ ]:
subs = dict()

# DO NOT SUBSTITUTE bulk comp, since this is defined in absolute comp
# subs.update([(ni,iX)  for ni, iX in zip(n, bulk_comp)])
# subs

In [ ]:

subs.update([(isym,imu) for isym, imu in zip(mu, mu_linear)])
subs[mu_shft] = 0
subs

In [ ]:
subs.update([(isym,ival) for isym, ival in zip(quad_consts, quad_coef)])

In [ ]:
# mu_expr.subs({[ni for ni in n:bulk_comp})
# d.update([(isym,imu) for isym, imu in zip(mu, mod_lstsq[:-1])])
# d[k_curv] = mod_lstsq[-1]
subs         

In [ ]:
# mu_local

In [ ]:
# mod_lstsq[:-1]

In [ ]:
# mu_local_mod = np.array(mu_expr.subs(subs), dtype=np.float).squeeze()
# mu_local_mod

In [ ]:
# mu_local_mod = np.array(mu_expr.subs(d), dtype=np.float).squeeze()
# dmu = mu_local-mu_local_mod
# mod_adj = np.hstack((mod_lstsq[:-1]+dmu, mod_lstsq[-1]))
# 
# yresid = np.dot(xobs, mod_adj)-yobs
# yobs.shape
# 
# np.dot(xobs, mod_lstsq)

In [ ]:
G

In [ ]:
# bulk_strs

In [ ]:
# bulk_sym

In [ ]:
# bulk composition not needed for global model

# for istr, isym in zip(bulk_strs, bulk_sym):
#     modelCD.add_expression_to_model(
#         G, [(istr, 'none', isym)])
    

modelCD.add_expression_to_model(G, [('mu_shft', 'J/m', mu_shft)])

In [ ]:
# modelCD.add_expression_to_model(G, [('k_curv', 'J/m^2', k_curv)])

In [ ]:
modelCD.module = "pseudo_soln"

In [ ]:
# dX = (X[:,0]-bulk_comp[:,np.newaxis])
# dX2 = np.dot(dX.T,dX)[0,0]
# dX2
# 
# G_mix, k_mix = sym.symbols('G_mix k_mix')

In [ ]:
# G_mix = k_mix*dX2
# G_mix

In [ ]:
# G = G_ss + G_mix

In [ ]:
# modelCD.add_expression_to_model(G, [('k_mix', 'none', k_mix)])

In [ ]:
# modelCD.module = "pseudo_soln"

In [ ]:
formula = ''
convert = []
test = []
if use_oxides_as_basis:
    for ind,elm in enumerate(elm_sys):
        ox_index = list(core.chem.oxide_props['oxides']).index(elm)
        ox_cat = core.chem.oxide_props['cations'][ox_index]
        formula += ox_cat + '[' + ox_cat + ']'
        ox_cat_num = core.chem.oxide_props['cat_num'][ox_index]
        if ox_cat_num > 1:
            convert.append('['+str(ind)+']=['+ox_cat+']/'+str(ox_cat_num)+'.0')
        else:
            convert.append('['+str(ind)+']=['+ox_cat+']')
        test.append('['+str(ind)+'] >= 0.0')
    formula += 'O[O]'
else:
    for ind,elm in enumerate(elm_sys):
        formula += elm + '[' + elm + ']'
        convert.append('['+str(ind)+']=['+elm+']')
        test.append('['+str(ind)+'] >= 0.0')
formula, convert, test

In [ ]:
modelCD.formula_string = formula
modelCD.conversion_string = convert
modelCD.test_string = test

In [ ]:
# mu_curv = np.tile(mu_curv/Ncross_terms, Ncross_terms)
# mu_curv

In [ ]:
# curv_local
# local_curv
quad_consts

In [ ]:
paramValues = {'T_r':298.15,'P_r':1.0}

In [ ]:
# for isym, ival in zip(quad_strs, mu_curv):
#     paramValues[isym] = ival
# paramValues['k_curv'] = mu_curv

# for ibulk_val, ibulk_str in zip(bulk_comp, bulk_strs):
#     paramValues[ibulk_str] = ibulk_val
    
# for ibulk_val, ibulk_str in zip(bulk_comp, bulk_strs):
#     paramValues[ibulk_str] = ibulk_val
    
paramValues['mu_shft'] = 0
    
endmembers = []
for elm in elm_sys:
    endmembers.append(str(elm)+'_pseudo_end')

In [ ]:
paramValues

In [ ]:
bulk_comp

In [ ]:
[iexp.expression for iexp in modelCD.expression_parts]

In [ ]:
paramValues

In [ ]:
# paramValues = {'k_mix':curv_local,'T_r':298.15,'P_r':1.0,}


In [ ]:
model_working_dir = "working"
!mkdir -p {model_working_dir}
%cd {model_working_dir}

In [ ]:
# cd ..

In [ ]:
modelCD.create_code_module(
    phase="PseudoPhase", params=paramValues, endmembers=endmembers, 
    prefix="cy", module_type='calib', silent=False)

In [ ]:
pwd

In [ ]:
import pseudo_soln
%cd ..

In [ ]:
endmembers

In [ ]:
G

In [ ]:
#%cd working
#import pseudo_soln
#%cd ..
modelPseudo = model.Database(database="CoderModule", calib="calib", 
                         phase_tuple=('pseudo_soln', {'Psu':['PseudoPhase','solution']}))
Pseudo = modelPseudo.get_phase('Psu')

for phase_name, abbrv in zip(modelPseudo.phase_info.phase_name,modelPseudo.phase_info.abbrev):
    print ('Abbreviation: {0:<10s} Name: {1:<30s}'.format(abbrv, phase_name))

In [ ]:
vals = Pseudo.get_param_values(all_params=True)
names = np.array(Pseudo.param_names)

# Pseudo.set_param_values(param_names=[2],param_values=[1.0])

In [ ]:
vals = Pseudo.get_param_values(all_params=True)
len(vals)

In [ ]:
mudiff = Pseudo.gibbs_energy(T, P, mol=bulk_comp)-mu_bulk

In [ ]:
mu_bulk

In [ ]:
Pseudo.endmember_names

In [ ]:
names

In [ ]:
# ind_kcurv = np.where(names=='k_curv')[0][0]
# k_curv
# Pseudo.set_param_values(param_names=[ind_kcurv], param_values=[0*local_curv[0]])

In [ ]:
np.sum(bulk_comp)

In [ ]:
# mu_bulk
mudiff/2

In [ ]:
# mu_shft_val=+50e3
# mu_shft_val=+10e3
# mu_shft_val=+15e3
# mu_shft_val=+100e3
# mu_shft_val=+1000e3
# mu_shft_val=-mu_bulk+5e3
mu_shft_val= +1169084.0 
Pseudo.set_param_values(param_names=[len(vals)-1], param_values=[mu_shft_val])
Pseudo.get_param_values(all_params=True)

In [ ]:
mudiff = Pseudo.gibbs_energy(T, P, mol=bulk_comp)-mu_bulk
mudiff

In [ ]:
Pseudo.get_param_values(all_params=True)

In [ ]:
# vals[2:] *= 10

In [ ]:
# mu_curv

In [ ]:
# vals

In [ ]:
# for ind, ival in enumerate(vals[2:]):
#     Pseudo.set_param_values(param_names=[ind+2], param_values=[ival])

In [ ]:
# Pseudo.get_param_values(all_params=True)

Check pseudo-phase import by printning some phase characteristics

In [ ]:
print (Pseudo.props['phase_name'])
print (Pseudo.props['formula'])
print (Pseudo.props['molwt'])
print (Pseudo.props['abbrev'])
print (Pseudo.props['endmember_num'])
print (Pseudo.props['endmember_name'])

## Try the equiibrium calculations with the omnicomponent pseudo-phase
#### Choose a phase assemblage

In [ ]:
#stix_phases.keys()
phs_sys  = [Pseudo]

# phs_sys.extend(phases.values())

# phs_sys += [phases['Fsp'], phases['Ol'], phases['Cpx'], phases['Grt']] # solutiopns,
# phs_sys += [phases['Qz'], phases['Ky'], phases['Nph']]
#
#phs_sys  = [Pseudo, stix_phases['Opx']]

In [ ]:
for iphssym in hull_phs_sym:
    phs_sys.append(phases[iphssym])
    
    

In [ ]:
phs_sys

In [ ]:
# phases = get_subsolidus_phases(database=database)

In [ ]:
sys_elems
# phs_sys

In [ ]:
# equil = equilibrate.Equilibrate(['O','Na','Mg','Al','Si','Ca','Fe'], phs_sys)
equil = equilibrate.Equilibrate(sys_elems, phs_sys)

In [ ]:
mu_bulk

In [ ]:
1467815.52

In [ ]:
state = equil.execute(T, P, bulk_comp=bulk_comp, debug=0)
state.print_state()